In [1]:
import pandas as pd
import numpy as np
import os


In [2]:
# Load data
with np.load(os.path.join('.', 'mnist-6k.npz')) as npz_file:
    X = npz_file['data']
    y = npz_file['labels']
    
print('X:', X.shape, X.dtype)
print('y:', y.shape, y.dtype)

X: (6000, 784) uint8
y: (6000,) int32
